In [2]:
import pandas as pd
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

In [3]:
all_data = pd.read_csv('data-ingest/data/beds_hb_wide_unfiltered.csv', parse_dates = [0])
all_data.columns = all_data.columns.str.lower()
all_data.head()

,date,health_board,co_gabed_c19,co_gabed_nonc19,co_gabed_spare,co_gabed_tot,co_ivbed_c19,co_ivbed_nonc19,co_ivbed_spare,co_ivbed_tot
0,2020-04-01,Betsi Cadwaladr University Health Board,93,656,499,1248,14,5,25,44
1,2020-04-01,Hywel Dda University Health Board,68,312,445,825,5,9,32,46
2,2020-04-01,Abertawe Bro Morgannwg University Health Board,100,515,423,1038,23,10,58,91
3,2020-04-01,Cardiff and Vale University Health Board,158,510,683,1351,19,13,31,63
4,2020-04-01,Cwm Taf University Health Board,111,486,353,950,16,7,34,57


In [4]:
lockdowns = pd.read_csv('data-ingest/data/lockdown_status_hb_date.csv')
lockdowns['date'] = pd.to_datetime(lockdowns['date'], format = '%Y-%m-%d')
lockdowns.head()
beds_lockdowns = all_data.merge(lockdowns[['date', 'health_board', 'lockdown_status']], how = 'inner', on = ['date', 'health_board'])
beds_lockdowns.head()

,date,health_board,co_gabed_c19,co_gabed_nonc19,co_gabed_spare,co_gabed_tot,co_ivbed_c19,co_ivbed_nonc19,co_ivbed_spare,co_ivbed_tot,lockdown_status
0,2020-04-01,Betsi Cadwaladr University Health Board,93,656,499,1248,14,5,25,44,No local lockdowns
1,2020-04-01,Hywel Dda University Health Board,68,312,445,825,5,9,32,46,No local lockdowns
2,2020-04-01,Abertawe Bro Morgannwg University Health Board,100,515,423,1038,23,10,58,91,No local lockdowns
3,2020-04-01,Cardiff and Vale University Health Board,158,510,683,1351,19,13,31,63,No local lockdowns
4,2020-04-01,Cwm Taf University Health Board,111,486,353,950,16,7,34,57,No local lockdowns


In [5]:
beds_lockdowns.columns = beds_lockdowns.columns.str.replace('co_', '')
beds_lockdowns.columns = beds_lockdowns.columns.str.replace('bed', '')
beds_lockdowns.head()

,date,health_board,ga_c19,ga_nonc19,ga_spare,ga_tot,iv_c19,iv_nonc19,iv_spare,iv_tot,lockdown_status
0,2020-04-01,Betsi Cadwaladr University Health Board,93,656,499,1248,14,5,25,44,No local lockdowns
1,2020-04-01,Hywel Dda University Health Board,68,312,445,825,5,9,32,46,No local lockdowns
2,2020-04-01,Abertawe Bro Morgannwg University Health Board,100,515,423,1038,23,10,58,91,No local lockdowns
3,2020-04-01,Cardiff and Vale University Health Board,158,510,683,1351,19,13,31,63,No local lockdowns
4,2020-04-01,Cwm Taf University Health Board,111,486,353,950,16,7,34,57,No local lockdowns


In [6]:
general_acute = beds_lockdowns.iloc[:, 0:6]
general_acute.head()

,date,health_board,ga_c19,ga_nonc19,ga_spare,ga_tot
0,2020-04-01,Betsi Cadwaladr University Health Board,93,656,499,1248
1,2020-04-01,Hywel Dda University Health Board,68,312,445,825
2,2020-04-01,Abertawe Bro Morgannwg University Health Board,100,515,423,1038
3,2020-04-01,Cardiff and Vale University Health Board,158,510,683,1351
4,2020-04-01,Cwm Taf University Health Board,111,486,353,950


In [7]:
beds_lockdowns.health_board.unique()

array(['Betsi Cadwaladr University Health Board',
       'Hywel Dda University Health Board',
       'Abertawe Bro Morgannwg University Health Board',
       'Cardiff and Vale University Health Board',
       'Cwm Taf University Health Board',
       'Aneurin Bevan University Health Board',
       'Velindre University NHS Trust', 'Wales',
       'Powys Teaching Health Board'], dtype=object)

In [8]:
# find the first day of local lockdowns for all healthboards.

# start with Abertawe so we can build a function and apply to all
def lockdown_countdown(df: pd.DataFrame, hb: str) -> pd.DataFrame:
    temp_df = df[df['health_board'] == hb].sort_values('date')
    first_day = temp_df[temp_df['lockdown_status'] != 'No local lockdowns'].iloc[0,0]
    temp_df = temp_df[temp_df['date']>= first_day - pd.Timedelta('28 days')]
    temp_df['daycount_from_local_lockdown'] = list(range(-28, len(temp_df)-28))
    return temp_df

countdown_df = pd.concat([lockdown_countdown(beds_lockdowns, hb) for hb in beds_lockdowns.health_board.unique() if hb not in ['Wales', 'Velindre University NHS Trust', 'Powys Teaching Health Board']])
countdown_df

,date,health_board,ga_c19,ga_nonc19,ga_spare,ga_tot,iv_c19,iv_nonc19,iv_spare,iv_tot,lockdown_status,daycount_from_local_lockdown
1373,2020-09-03,Betsi Cadwaladr University Health Board,71,1504,344,1919,0,21,29,50,No local lockdowns,-28
1382,2020-09-04,Betsi Cadwaladr University Health Board,66,1515,339,1920,0,27,23,50,No local lockdowns,-27
1391,2020-09-05,Betsi Cadwaladr University Health Board,68,1558,260,1886,1,25,24,50,No local lockdowns,-26
1400,2020-09-06,Betsi Cadwaladr University Health Board,70,1607,226,1903,0,27,23,50,No local lockdowns,-25
1409,2020-09-07,Betsi Cadwaladr University Health Board,81,1591,236,1908,0,25,25,50,No local lockdowns,-24
...,...,...,...,...,...,...,...,...,...,...,...,...
2035,2020-11-15,Aneurin Bevan University Health Board,425,763,490,1678,9,9,17,35,Post-firebreak,68
2044,2020-11-16,Aneurin Bevan University Health Board,441,763,905,2109,10,7,10,27,Post-firebreak,69
2053,2020-11-17,Aneurin Bevan University Health Board,447,827,873,2147,11,2,11,24,Post-firebreak,70
2062,2020-11-18,Aneurin Bevan University Health Board,436,858,853,2147,14,3,7,24,Post-firebreak,71


In [9]:
countdown_df['total_covid'] = countdown_df['ga_c19'] + countdown_df['iv_c19']
countdown_df['total_beds'] = countdown_df['ga_tot'] + countdown_df['iv_tot']
countdown_df['percent_in_use'] = round(countdown_df['total_covid'] / countdown_df['total_beds'] * 100, 2)
countdown_df.head()

,date,health_board,ga_c19,ga_nonc19,ga_spare,ga_tot,iv_c19,iv_nonc19,iv_spare,iv_tot,lockdown_status,daycount_from_local_lockdown,total_covid,total_beds,percent_in_use
1373,2020-09-03,Betsi Cadwaladr University Health Board,71,1504,344,1919,0,21,29,50,No local lockdowns,-28,71,1969,3.61
1382,2020-09-04,Betsi Cadwaladr University Health Board,66,1515,339,1920,0,27,23,50,No local lockdowns,-27,66,1970,3.35
1391,2020-09-05,Betsi Cadwaladr University Health Board,68,1558,260,1886,1,25,24,50,No local lockdowns,-26,69,1936,3.56
1400,2020-09-06,Betsi Cadwaladr University Health Board,70,1607,226,1903,0,27,23,50,No local lockdowns,-25,70,1953,3.58
1409,2020-09-07,Betsi Cadwaladr University Health Board,81,1591,236,1908,0,25,25,50,No local lockdowns,-24,81,1958,4.14


In [10]:
beds_stacked = countdown_df[['daycount_from_local_lockdown', 'health_board', 'ga_c19', 'ga_nonc19', 'ga_spare']].pivot(index = 'daycount_from_local_lockdown', columns = 'health_board', values = ['ga_c19', 'ga_nonc19', 'ga_spare'])
beds_stacked.head()

ga_c19  \
health_board                 Abertawe Bro Morgannwg University Health Board   
daycount_from_local_lockdown                                                  
-28                                                                     9.0   
-27                                                                     7.0   
-26                                                                     9.0   
-25                                                                    14.0   
-24                                                                     5.0   

                                                                    \
health_board                 Aneurin Bevan University Health Board   
daycount_from_local_lockdown                                         
-28                                                           40.0   
-27                                                           36.0   
-26                                                           41.0   
-25                                                           41.0   
-24                                                          100.0   

                                                                      \
health_board                 Betsi Cadwaladr University Health Board   
daycount_from_local_lockdown                                           
-28                                                             71.0   
-27                                                             66.0   
-26                                                             68.0   
-25                                                             70.0   
-24                                                             81.0   

                                                                       \
health_board                 Cardiff and Vale University Health Board   
daycount_from_local_lockdown                                            
-28                                                              72.0   
-27                                                              79.0   
-26                                                              65.0   
-25                                                              70.0   
-24                                                             101.0   

                                                              \
health_board                 Cwm Taf University Health Board   
daycount_from_local_lockdown                                   
-28                                                     41.0   
-27                                                     37.0   
-26                                                     36.0   
-25                                                     38.0   
-24                                                     40.0   

                                                                \
health_board                 Hywel Dda University Health Board   
daycount_from_local_lockdown                                     
-28                                                       18.0   
-27                                                       17.0   
-26                                                       17.0   
-25                                                       11.0   
-24                                                       10.0   

                                                                  ga_nonc19  \
health_board                 Abertawe Bro Morgannwg University Health Board   
daycount_from_local_lockdown                                                  
-28                                                                  1061.0   
-27                                                                  1095.0   
-26                                                                  1095.0   
-25                                                                  1107.0   
-24                                                                  1143.0   

                                      

In [11]:
# show the specific traces you allow.

fig = go.Figure()

for column in [y for _, y in beds_stacked.iloc[:, 0:6].columns]:
    fig.add_trace(
        go.Bar(
            x = beds_stacked.index,
            y = beds_stacked[('ga_c19', column)],
            name = column + '_c19'
        )
    )
    fig.add_trace(
        go.Bar(
            x = beds_stacked.index,
            y = beds_stacked[('ga_nonc19', column)],
            name = column + '_nonc19'
        ))
    fig.add_trace(
        go.Bar(
            x = beds_stacked.index,
            y = beds_stacked[('ga_spare', column)],
            name = column + '_spare'
        ))
    
base_title = 'Beds in use in health board: '

fig.update_layout(
    barmode = 'stack',
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'Abertawe Bro Morgannwg University Health Board',
                  method = 'update',
                  args = [{'visible': [True, True, True] +  [False, False, False, False, False] * 3},
                          {'title': base_title + 'Abertawe Bro Morgannwg',
                           'showlegend':True}]),
             dict(label = 'Aneurin Bevan University Health Board',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, True, True] + [False, False, False, False] * 3}, 
                          {'title': base_title + 'Aneurin Bevan University Health Board',
                           'showlegend':True}]),
             dict(label = 'Betsi Cadwaladr University Health Board',
                  method = 'update',
                  args = [{'visible': [False, False] * 3 + [True, True, True] + [False, False, False] * 3},
                          {'title': base_title + 'Betsi Cadwaladr University Health Board',
                           'showlegend':True}]),
             dict(label = 'Cardiff and Vale University Health Board',
                  method = 'update',
                  args = [{'visible': [False, False, False] * 3 + [True, True, True] +  [False, False] * 3},
                          {'title': base_title + 'Cardiff and Vale University Health Board',
                           'showlegend':True}]),
             dict(label = 'Cwm Taf University Health Board',
                  method = 'update',
                  args = [{'visible': [False, False, False, False] * 3 + [True, True, True, False, False, False]},
                          {'title': base_title + 'Cwm Taf University Health Board',
                           'showlegend':True}]),
             dict(label = 'Hywel Dda University Health Board',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False] * 3 + [True, True, True]},
                          {'title': base_title + 'Hywel Dda University Health Board',
                           'showlegend':True}]),
            ])
        )
    ])

fig.show()

In [12]:
hosps_admissions = pd.read_csv('data-ingest/data/admissions_hospitalisations_change.csv', parse_dates = [0])
hosps_admissions.head()

,date,health_board,Suspected,Confirmed,Recovering,total_hospitalisations,new_admissions,prev_day_beds_occupied,difference
0,2020-04-01,Abertawe Bro Morgannwg University Health Board,74,49,NaN,123,20,NaN,NaN
1,2020-04-02,Abertawe Bro Morgannwg University Health Board,81,59,NaN,140,25,123.0,-8.0
2,2020-04-03,Abertawe Bro Morgannwg University Health Board,77,72,NaN,149,23,140.0,-14.0
3,2020-04-04,Abertawe Bro Morgannwg University Health Board,65,78,NaN,143,24,149.0,-30.0
4,2020-04-05,Abertawe Bro Morgannwg University Health Board,68,105,NaN,173,15,143.0,15.0


In [13]:
daily_admissions = hosps_admissions.drop(columns = ['Suspected', 'Confirmed', 'Recovering', 'total_hospitalisations', 'prev_day_beds_occupied', 'difference'])
daily_admissions['daily_change_percent'] = round((daily_admissions['new_admissions'] - daily_admissions.shift(1)['new_admissions']) / daily_admissions.shift(1)['new_admissions'] * 100, 2)
daily_admissions['ravg_admissions'] = daily_admissions.groupby('health_board')['new_admissions'].transform(lambda x: x.rolling(7, 1).mean())
daily_admissions.head()

,date,health_board,new_admissions,daily_change_percent,ravg_admissions
0,2020-04-01,Abertawe Bro Morgannwg University Health Board,20,NaN,20.000000
1,2020-04-02,Abertawe Bro Morgannwg University Health Board,25,25.00,22.500000
2,2020-04-03,Abertawe Bro Morgannwg University Health Board,23,-8.00,22.666667
3,2020-04-04,Abertawe Bro Morgannwg University Health Board,24,4.35,23.000000
4,2020-04-05,Abertawe Bro Morgannwg University Health Board,15,-37.50,21.400000


In [14]:
admissions_lockdowns = beds_lockdowns = daily_admissions.merge(lockdowns[['date', 'health_board', 'lockdown_status']], how = 'inner', on = ['date', 'health_board'])

admissions_lockdowns_countdown = pd.concat([lockdown_countdown(admissions_lockdowns, hb) for hb in admissions_lockdowns.health_board.unique() if hb not in ['Wales', 'Velindre University NHS Trust', 'Powys Teaching Health Board']])
admissions_lockdowns_countdown

,date,health_board,new_admissions,daily_change_percent,ravg_admissions,lockdown_status,daycount_from_local_lockdown
151,2020-08-30,Abertawe Bro Morgannwg University Health Board,2,-71.43,3.000000,No local lockdowns,-28
152,2020-08-31,Abertawe Bro Morgannwg University Health Board,1,-50.00,2.857143,No local lockdowns,-27
153,2020-09-01,Abertawe Bro Morgannwg University Health Board,2,100.00,2.857143,No local lockdowns,-26
154,2020-09-02,Abertawe Bro Morgannwg University Health Board,8,300.00,3.714286,No local lockdowns,-25
155,2020-09-03,Abertawe Bro Morgannwg University Health Board,1,-87.50,3.571429,No local lockdowns,-24
...,...,...,...,...,...,...,...
1387,2020-11-14,Hywel Dda University Health Board,16,220.00,12.000000,Post-firebreak,49
1388,2020-11-15,Hywel Dda University Health Board,8,-50.00,10.142857,Post-firebreak,50
1389,2020-11-16,Hywel Dda University Health Board,8,0.00,10.000000,Post-firebreak,51
1390,2020-11-17,Hywel Dda University Health Board,9,12.50,8.714286,Post-firebreak,52


In [15]:
fig = px.line(admissions_lockdowns_countdown, x = 'daycount_from_local_lockdown', y = 'daily_change_percent', color = 'health_board')
fig.show()

In [16]:
admissions_lockdowns_countdown.lockdown_status.unique()

array(['No local lockdowns', 'Local lockdowns', 'Firebreak lockdown',
       'Post-firebreak'], dtype=object)

In [17]:
first_day_firebreak = admissions_lockdowns_countdown[admissions_lockdowns_countdown['lockdown_status']=='Firebreak lockdown'].groupby('health_board')['daycount_from_local_lockdown'].first()
first_day_firebreak

health_board
Abertawe Bro Morgannwg University Health Board    26
Aneurin Bevan University Health Board             45
Betsi Cadwaladr University Health Board           22
Cardiff and Vale University Health Board          26
Cwm Taf University Health Board                   36
Hywel Dda University Health Board                 27
Name: daycount_from_local_lockdown, dtype: int64

In [18]:
first_day_local = admissions_lockdowns_countdown[admissions_lockdowns_countdown['daycount_from_local_lockdown']==0][['health_board', 'date']].set_index('health_board')
first_day_local

,date
health_board,
Abertawe Bro Morgannwg University Health Board,2020-09-27
Aneurin Bevan University Health Board,2020-09-08
Betsi Cadwaladr University Health Board,2020-10-01
Cardiff and Vale University Health Board,2020-09-27
Cwm Taf University Health Board,2020-09-17
Hywel Dda University Health Board,2020-09-26


In [19]:
ravg_healthboards = admissions_lockdowns_countdown[admissions_lockdowns_countdown['daycount_from_local_lockdown'] <= 50].pivot(index = 'daycount_from_local_lockdown', columns = 'health_board', values = 'ravg_admissions')

In [20]:
# how long did the local lockdowns last?
durations = {}
for column in ravg_healthboards.columns.to_list():
    durations[column] = first_day_firebreak.loc[column]
    
durations

{'Abertawe Bro Morgannwg University Health Board': 26,
 'Aneurin Bevan University Health Board': 45,
 'Betsi Cadwaladr University Health Board': 22,
 'Cardiff and Vale University Health Board': 26,
 'Cwm Taf University Health Board': 36,
 'Hywel Dda University Health Board': 27}

In [22]:
fig = go.Figure()

colors = ['#747875', '#9C528B', '#252637', '#50869D', '#ED8103', '#794101']
colors_columns = {col: color for col, color in zip(ravg_healthboards.columns.to_list(), colors)}
cols_of_interest = ['Aneurin Bevan University Health Board', 'Betsi Cadwaladr University Health Board', 'Cwm Taf University Health Board']
showlegend = True

# add dot to show where firebreak lockdown began
for column in ravg_healthboards.columns.to_list():
    opacity = 1 if column in cols_of_interest else 0.4
    lockdown_dur = durations[column]
    fig.add_trace(
        go.Scatter(
            name = "firebreak lockdown",
            x = [first_day_firebreak.loc[column]],
            y = [ravg_healthboards.loc[first_day_firebreak.loc[column], column]],
            mode = 'markers',
            marker = dict(color = colors_columns[column], size = 8),
            showlegend=showlegend,
            opacity = opacity,
            hovertemplate = f"Firebreak lockdown begins in <br>{column} <br>after {lockdown_dur} days of local lockdowns"
        )
    )
    showlegend = False

# add trace for each health board
for column in ravg_healthboards.columns.to_list():
    opacity = 1 if column in cols_of_interest else 0.4
    x = ravg_healthboards.index
    y = ravg_healthboards[column]
    fig.add_trace(
        go.Scatter(
            x = x,
            y = y,
            name = column,
            line = dict(color = colors_columns[column]),
            opacity = opacity,
            hovertemplate = 'Days since start of local lockdowns: %{x}<br>Hospital admissions rolling average: %{y:.2f}'
        )
    )
    
# add dot so that tooltip will display date of first lockdown
for column in ravg_healthboards.columns.to_list():
    local_loc_date = first_day_local.loc[column, "date"].strftime("%Y-%m-%d")
    fig.add_trace(
        go.Scatter(
            name = "Lockdown begins",
            x = [0],
            y = [ravg_healthboards.loc[0, column]],
            mode = 'markers',
            marker = dict(color = colors_columns[column], size = 1),
            showlegend=False,
            opacity = 0.0,
            hovertemplate = f'Local lockdowns begins on {local_loc_date}' + '<br>Hospital admissions rolling average: %{y:.2f}'
        )
    )
    
base_title = "Weekly rolling average of daily hospital admissions<br>Health Board: "

# Add drop down menu
fig.update_layout(
    updatemenus=[go.layout.Updatemenu(
        active=0,
        buttons=list(
            [dict(label = 'All',
                  method = 'update',
                  args = [{'visible': [True, True, True, True, True, True] * 2}, # all plots
                          {'title': base_title + 'All',
                           'showlegend':True}]),
             dict(label = 'Abertawe Bro Morgannwg',
                  method = 'update',
                  args = [{'visible': [True, False, False, False,False, False] * 2}, # the index of True aligns with the indices of plot traces
                          {'title': base_title + 'Abertawe Bro Morgannwg',
                           'showlegend':True}]),
             dict(label = 'Aneurin Bevan University Health Board',
                  method = 'update',
                  args = [{'visible': [False, True, False, False,False, False] * 2},
                          {'title': base_title + 'Aneurin Bevan',
                           'showlegend':True}]),
            dict(label = 'Betsi Cadwaladr',
                  method = 'update',
                  args = [{'visible': [False, False, True, False, False, False] * 2},
                          {'title': base_title + 'Betsi Cadwaladr',
                           'showlegend':True}]),
             dict(label = 'Cardiff and Vale University Health Board',
                  method = 'update',
                  args = [{'visible': [False, False, False, True, False, False] * 2},
                          {'title': base_title + 'Cardiff and Vale',
                           'showlegend':True}]),
             dict(label = 'Cwm Taf',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, True, False] * 2},
                          {'title': base_title + 'Cwm Taf',
                           'showlegend':True}]),
             dict(label = 'Hywel Dda',
                  method = 'update',
                  args = [{'visible': [False, False, False, False, False, True] * 2},
                          {'title': base_title + 'Cwm Taf',
                           'showlegend':True}]),
            ])
        )
    ])

# add vertical lines to show beginning of lockdown and day 12
for lab, pos in [('Local lockdowns begin', 0), ('Day 12*', 12)]:
    fig.add_shape(type="line",
        x0=pos, y0=0, x1=pos, y1=35,
        line=dict(
            color="Gray",
            width=2,
            dash="dash",
        ),
        opacity = 0.6
    )
    # add labels
    fig.add_annotation(
        x=pos-0.5, y=25, 
        text=lab, 
        showarrow=False, 
        xanchor = 'right', yanchor = 'bottom'
    )
    
fig.add_annotation(
    x = 30, y = 4,
    xanchor = 'left', yanchor = 'top',
    showarrow = False,
    text = "* 12 is the median number of days <br> between infection and hospitalisation<br>(Wang et al., 2020 & CDC, 2020)",
    bgcolor = 'gray',
    font = dict(color = 'white'),
    opacity = 0.5,
    align = 'left'
)

# update axes
fig.update_layout(plot_bgcolor='rgba(0,0,0,0)')

fig.write_html('visualisations/ravg_admissions_hb.html')
fig.show()